### **Libraries**

In [ ]:
 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import numpy as np
from tensorflow.keras import optimizers
import tensorflow.keras as keras
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install kaggle
!pip install --upgrade kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c uw-cs480-fall20
!ls
import zipfile
zip_ref = zipfile.ZipFile('uw-cs480-fall20.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()

     |████████████████████████████████| 61kB 3.3MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.10-cp36-none-any.whl size=73269 sha256=f0bee0a1e5e28dffc4f00cd7a3f671a80e27db733837208a7510e3989e6921e9
  Stored in directory: /root/.cache/pip/wheels/3a/d1/7e/6ce09b72b770149802c653a02783821629146983ee5a360f10
Successfully built kaggle
  Found existing installation: kaggle 1.5.9
    Uninstalling kaggle-1.5.9:
      Successfully uninstalled kaggle-1.5.9


Saving kaggle.json to kaggle.json
 85% 73.0M/85.6M [00:06<00:01, 9.04MB/s]
100% 85.6M/85.6M [00:06<00:00, 13.3MB/s]
drive  kaggle.json  sample_data  uw-cs480-fall20.zip


### **Data Loading**

In [ ]:
import pandas as pd
train = pd.read_csv('/content/files/train.csv',dtype=str)
test = pd.read_csv('/content/files/test.csv',dtype=str)
# train=train.reset_index(drop=True)
# test=test.reset_index(drop=True)
def Addjpg(fn):
    return fn+".jpg"
train["id"]=train["id"].apply(Addjpg)
test["id"]=test["id"].apply(Addjpg)   
train_folder = '/content/files/suffled-images/shuffled-images'
test_folder = '/content/files/suffled-images/shuffled-images'

In [ ]:
# Define data generator
train_gen = ImageDataGenerator(
validation_split=0.05,
rotation_range=45,
rescale=1./255,
horizontal_flip=True)
test_gen = ImageDataGenerator(rescale = 1./255)


In [ ]:
train_data = train_gen.flow_from_dataframe(dataframe = train, 
directory = train_folder, x_col = 'id', 
y_col = 'category', subset="training", seed = 42,
batch_size = 100, shuffle = True, 
class_mode="categorical",target_size = (80, 60))

validation_data = train_gen.flow_from_dataframe(dataframe = train, 
directory = train_folder, x_col = 'id', 
y_col = 'category', subset="validation", seed = 42,
batch_size = 100, shuffle = True, 
class_mode="categorical",target_size = (80, 60))

test_data = test_gen.flow_from_dataframe(dataframe = test, 
directory = test_folder, x_col = 'id', 
y_col = None,
batch_size = 100, shuffle = False, 
class_mode=None,target_size = (80, 60))

Found 20546 validated image filenames belonging to 27 classes.
Found 1081 validated image filenames belonging to 27 classes.
Found 21628 validated image filenames.


### **Image VGG16**

In [ ]:
model = Sequential()
model.add(Conv2D(input_shape=(60,80,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=27, activation="softmax"))

In [ ]:
model.compile(tf.keras.optimizers.RMSprop(lr=0.0001,decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
STEP_SIZE_TRAIN=train_data.n//train_data.batch_size
STEP_SIZE_VALID=validation_data.n//validation_data.batch_size
STEP_SIZE_TEST=test_data.n//test_data.batch_size
model.fit_generator(generator=train_data,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_data,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50)

Epoch 1/50
205/205 [==============================] - 38s 185ms/step - loss: 2.3240 - accuracy: 0.3845 - val_loss: 1.9282 - val_accuracy: 0.5090
Epoch 2/50
205/205 [==============================] - 37s 182ms/step - loss: 1.5510 - accuracy: 0.5737 - val_loss: 1.3237 - val_accuracy: 0.6180
Epoch 3/50
205/205 [==============================] - 37s 182ms/step - loss: 1.1101 - accuracy: 0.6817 - val_loss: 0.9146 - val_accuracy: 0.7210
Epoch 4/50
205/205 [==============================] - 37s 183ms/step - loss: 0.8799 - accuracy: 0.7425 - val_loss: 0.8266 - val_accuracy: 0.7730
Epoch 5/50
205/205 [==============================] - 38s 183ms/step - loss: 0.7384 - accuracy: 0.7836 - val_loss: 0.6626 - val_accuracy: 0.8040
Epoch 6/50
205/205 [==============================] - 38s 185ms/step - loss: 0.6364 - accuracy: 0.8136 - val_loss: 0.6035 - val_accuracy: 0.8230
Epoch 7/50
205/205 [==============================] - 38s 183ms/step - loss: 0.5712 - accuracy: 0.8343 - val_loss: 0.5345 - val_ac

In [ ]:
test_data.reset()
pred_vgg=model.predict_generator(test_data,verbose=1)
predicted_class_indices_vgg=np.argmax(pred_vgg,axis=1)
labels = (train_data.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions_vgg = [labels[k] for k in predicted_class_indices_vgg]
filenames=test_data.filenames
results_vgg=pd.DataFrame({"id":filenames,"category":predictions_vgg})
results_vgg.to_csv("results_rmsp_vgg.csv",index=False)

In [ ]:
model.save('my_model_92.h2')
model2 = keras.models.load_model('my_model_92.h2')
print(model2.predict_generator(test_data,verbose=1))

Code Libarary (useful commands)

In [ ]:
# le = preprocessing.LabelEncoder()

# # print(train["baseColour"])
# print(type(train))
# a=train["gender"]
# # a = train[['gender', 'season']] 
# print(a)
# b=pd.Categorical(a)
# print(b)
# le.fit(b)
# print(le.classes_)
# c=le.transform(b)
# print(type(c))
# print(c)
# print(type(b))

# # #############################
# print(train["noisyTextDescription"])


# print(test_data[215][99][79][59][2])
# print(train_data[215][1].shape)
# print(train_data[215][0].shape)